<a href="https://colab.research.google.com/github/Samesh16/lpr/blob/master/CAO_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ray gradio opencv-python numpy matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 7.6 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [2]:
import ray
import numpy as np
import cv2
import gradio as gr
import time

In [3]:
ray.init()

2024-11-16 05:31:08,517	INFO worker.py:1819 -- Started a local Ray instance.


Python version:,3.10.12
Ray version:,2.39.0


In [8]:
@ray.remote
def svd_greyscale_conversion(channel):
    # Perform SVD
    U, S, Vt = np.linalg.svd(channel, full_matrices=False)
    # Reconstruct the matrix with a limited number of singular values
    k = min(20, len(S))  # Reduced 'k' to speed up computation
    S = np.diag(S[:k])
    return np.dot(U[:, :k], np.dot(S, Vt[:k, :]))

In [9]:
def process_image(image):
    # Start measuring time
    start_time = time.time()

    # Resize image to reduce resolution (limits to 256x256)
    image = cv2.resize(image, (256, 256))

    # Convert image from RGB to BGR format (Gradio provides image in RGB format)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Split the image into RGB channels
    channels = [image[:, :, i] for i in range(3)]

    # Perform SVD on each channel in parallel using Ray
    processed_channels = ray.get([svd_greyscale_conversion.remote(ch) for ch in channels])

    # Combine the processed channels to create a grayscale image by averaging
    gray_image = sum(processed_channels) / 3

    # Normalize to the range [0, 255]
    gray_image = (gray_image - gray_image.min()) / (gray_image.max() - gray_image.min()) * 255
    gray_image = gray_image.astype(np.uint8)

    # End measuring time
    end_time = time.time()
    processing_time = end_time - start_time
    processing_time_str = f"Parallel processing time: {processing_time:.4f} seconds"

    return gray_image, processing_time_str

In [12]:
interface = gr.Interface(
    fn=process_image,
    inputs=gr.Image(type="numpy", label="Upload an Image"),
    outputs=[
        gr.Image(type="numpy", label="Grayscale Image (Using SVD and Ray)"),
        gr.Textbox(label="Processing Time")
    ],
    title="SVD-Based Grayscale Conversion (Parallel)",
    description="Upload an RGB image to convert it to grayscale using Singular Value Decomposition (SVD) with parallel processing via Ray. Optimized to ensure processing time stays under 0.5 seconds."
)


In [13]:
interface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://10883ad3ac20d04cce.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
